In [ ]:
import numpy as np
import argparse
import time

In [ ]:
# This cell is from https://gist.github.com/akesling/5358964 and provides functions to 
# read MNIST into numpy arrays for processing. I did not write this.
import os
import struct

"""
Loosely inspired by http://abel.ee.ucla.edu/cvxopt/_downloads/mnist.py
which is GPL licensed.
"""

def read(dataset = "training", path = "."):
    """
    Python function for importing the MNIST data set.  It returns an iterator
    of 2-tuples with the first element being the label and the second element
    being a numpy.uint8 2D array of pixel data for the given image.
    """

    if dataset is "training":
        fname_img = os.path.join(path, 'train-images-idx3-ubyte')
        fname_lbl = os.path.join(path, 'train-labels-idx1-ubyte')
        print(fname_img)
    elif dataset is "testing":
        fname_img = os.path.join(path, 't10k-images-idx3-ubyte')
        fname_lbl = os.path.join(path, 't10k-labels-idx1-ubyte')
    else:
        raise ValueError("dataset must be 'testing' or 'training'")

    # Load everything in some numpy arrays
    with open(fname_lbl, 'rb') as flbl:
        magic, num = struct.unpack(">II", flbl.read(8))
        lbl = np.fromfile(flbl, dtype=np.int8)

    with open(fname_img, 'rb') as fimg:
        magic, num, rows, cols = struct.unpack(">IIII", fimg.read(16))
        img = np.fromfile(fimg, dtype=np.uint8).reshape(len(lbl), rows, cols)

    get_img = lambda idx: (lbl[idx], img[idx])

    # Create an iterator which returns each image in turn
    for i in range(len(lbl)):
        yield get_img(i)

def show(image):
    """
    Render a given numpy.uint8 2D array of pixel data.
    """
    from matplotlib import pyplot
    import matplotlib as mpl
    fig = pyplot.figure()
    ax = fig.add_subplot(1,1,1)
    imgplot = ax.imshow(image, cmap=mpl.cm.Greys)
    imgplot.set_interpolation('nearest')
    ax.xaxis.set_ticks_position('top')
    ax.yaxis.set_ticks_position('left')
    pyplot.show()

In [ ]:
training_data_orig = list(read(dataset='training', path='.'))
print(len(training_data_orig))
label, pixels = training_data_orig[420]
print(label)
print(pixels.shape)
show(pixels)

In [ ]:
def binarize(pic):
    temp = np.zeros(np.shape(pic))
    for row in range(len(pic)):
        for col in range(len(pic[0])):
            if pic[row][col] < (255.0 / 2):
                temp[row][col] = -1
            else: 
                temp[row][col] = 1
    return temp

In [ ]:
training_data = training_data_orig[:500]
training_binarized = [(label, pixel) for label, pixel in training_data]
for i in range(len(training_data)):
    asdf = binarize(training_data[i][1])
    training_binarized[i] = (label, asdf)
# print(training_binarized)

In [ ]:
training_noisy = training_binarized.copy()
# flip 16 values in each image
for i in range(len(training_binarized)):
    for count in range(16):
        row = np.random.randint(0,28)
        col = np.random.randint(0,28)
        training_noisy[i][1][row][col] = -1 * training_binarized[i][1][row][col]

In [ ]:
theta_hi_xj = 2
theta_hi_hj = 0.2
def boltzmann(i):
    expected = 1 # Expected value of H_i and X_i given theta
    numer = np.exp(-expected)
    denom = np.exp()
    return numer / denom